In [9]:
import json
import numpy as np
import pandas as pd
import regex
import os

In [2]:
def get_tracing(path):
    with open(path, 'r') as f:
        json_data = json.load(f)
    columns=["Thread","Time","Event"]
    result_df=pd.DataFrame(columns=columns)
    def _get_df_row(term):
        df_row={}
        if term["name"] == "Initialize" and term["pid"] == "All":
            df_row["Thread"]= term["pid"]
            df_row["Time"]= term["ts"]
            df_row["Event"]= "Start"
        elif term["name"] == "Finalize" and term["pid"] == "All":
            df_row["Thread"]= term["pid"]
            df_row["Time"]= term["ts"]
            df_row["Event"]= "End"
        elif term["tid"] == "Parallel Region" and term["pid"] == "All":
            df_row["Thread"]= term["pid"]
            df_row["Time"]= term["ts"]
            df_row["Event"]= term["tid"]
        else:
            thread_id = int(term["pid"].split(" ")[1])
            df_row["Thread"]= thread_id
            df_row["Time"]= term["ts"]
            df_row["Event"]= term["tid"]
        return df_row
    for term,i in zip(json_data,range(len(json_data))):
        result_df.loc[i]=_get_df_row(term)
    return result_df

In [3]:
def thread_ratio(df):
    df=df[df["Event"]=="Thread Region"]
    df = df.sort_values(by=["Thread","Time"])
    df = df.reset_index(drop=True)
    num_threads=np.sort(np.int64(df["Thread"].unique()))
    def _thread_time(df,i):
        tmp_df=df[df["Thread"]==i]
        tmp_df.reset_index(drop=True)
        tmp_time=0
        assert len(tmp_df)%2==0
        for j in range(0,len(tmp_df),2):
            tmp_time+=tmp_df.iloc[j+1]["Time"]-tmp_df.iloc[j]["Time"]
        return tmp_time
    seq_time=_thread_time(df,0)
    worker_time=[]
    for i in num_threads[1:]:
        worker_time.append(_thread_time(df,i))
    return max(worker_time)/seq_time

In [4]:
def sync_ratio(df):
    run_time=df[df["Event"]=="End"]["Time"].values[0]-df[df["Event"]=="Start"]["Time"].values[0]
    df=df[df["Event"]=="Sync Region"]
    df = df.sort_values(by=["Thread","Time"])
    df = df.reset_index(drop=True)
    num_threads=np.sort(np.int64(df["Thread"].unique()))
    def _thread_time(df,i):
        tmp_df=df[df["Thread"]==i]
        tmp_df.reset_index(drop=True)
        tmp_time=0
        assert len(tmp_df)%2==0
        for j in range(0,len(tmp_df),2):
            tmp_time+=tmp_df.iloc[j+1]["Time"]-tmp_df.iloc[j]["Time"]
        return tmp_time
    sync_time=[]
    for i in num_threads:
        sync_time.append(_thread_time(df,i))
    return max(sync_time)/run_time

In [5]:
def parallel_region_ratio(df):
    run_time=df[df["Event"]=="End"]["Time"].values[0]-df[df["Event"]=="Start"]["Time"].values[0]
    tmp_df=df[(df["Thread"]=="All") & (df["Event"]=="Parallel Region")]
    tmp_df=tmp_df.sort_values(by=["Time"])
    assert(tmp_df.shape[0]%2==0)
    parallel_time=0
    for i in range(0,tmp_df.shape[0],2):
        start_time=tmp_df.iloc[i]["Time"]
        end_time=tmp_df.iloc[i+1]["Time"]
        parallel_time+=end_time-start_time
    return parallel_time/run_time

In [6]:
def get_valgrind(path):
    # path="/home/yiwei/Bottleneck-Predictions-of-Parallel-Programs/valgrind_sample.txt"
    logfile=open(path,"r")
    line_list=logfile.readlines()
    line_len=len(line_list)
    columns=["Ir","I1mr","ILmr","Dr","D1mr","DLmr","Dw","D1mw","DLmw","Function"]
    df_counts=pd.DataFrame(columns=columns,dtype='object')
    df_ratios=pd.DataFrame(columns=columns,dtype='object')
    # The stats of all data
    head_index=0
    for line_index in range(line_len):
        m=regex.match(r"Ir                 I1mr           ILmr           Dr               D1mr           DLmr           Dw               D1mw           DLmw",line_list[line_index])
        if m!=None:
            head_index=line_index
            break
    assert head_index!=0
    
    def _read_valgrind_line(line):
        tmp_list=regex.split(r"\(|\)| ",line)
        tmp_list = list(filter(None, tmp_list))
        if len(tmp_list)<10:
            return None,None,None
        counts=[]
        ratios=[]
        k=0
        for i in range(9):
            counts.append(int(tmp_list[k].replace(",","")))
            k=k+1
            tmp_ratio=tmp_list[k]
            if tmp_ratio[-1]=="%":
                ratios.append(float(tmp_ratio[:-1])/100)
                k=k+1
            else:
                ratios.append(0.0)
        if len(counts)==0:
            return None,None,None
        func=tmp_list[-1].replace("\n","")
        if len(func)>0 and func[0]==".":
            func=None
        return counts,ratios,func
    
    counts,ratios,func=_read_valgrind_line(line_list[head_index+2])
    df_counts.loc[0]=pd.Series(counts,index=columns[:-1],dtype='object')
    df_counts.loc[0]["Function"]=func
    df_ratios.loc[0]=pd.Series(ratios,index=columns[:-1],dtype='object')
    df_ratios.loc[0]["Function"]=func

    # The stats of functions
    begin_index=head_index+2
    for line_index in range(head_index+2,line_len):
        m=regex.match(r"Ir                 I1mr         ILmr         Dr               D1mr         DLmr         Dw              D1mw           DLmw          file:function",line_list[line_index])
        if m!=None:
            begin_index=line_index
            break
    assert begin_index!=0


    for line_index in range(begin_index+3,line_len):
    # for line_index in [23]:
        counts,ratios,func=_read_valgrind_line(line_list[line_index])
        if counts==None:
            end_index=line_index
            break
        else:
            k=len(df_counts)
            df_counts.loc[k]=pd.Series(counts,index=columns[:-1],dtype='object')
            df_counts.loc[k]["Function"]=func
            df_ratios.loc[k]=pd.Series(ratios,index=columns[:-1],dtype='object')
            df_ratios.loc[k]["Function"]=func
    return df_counts,df_ratios


In [7]:
test_case = "./test_imbalance/"

In [32]:
def get_ratio_sync_paral(test_case):
    """ input test_case: the path of the test case"""
    jsonfiles = [file for file in os.listdir(test_case) if file.endswith(".json")]
    ratios = []
    for jsonfile in jsonfiles:
        print(jsonfile)
        result = get_tracing(os.path.join(test_case, jsonfile))
        parallel_ratio = parallel_region_ratio(result)
        s_ratio = sync_ratio(result)
        print("parllel region ratio:", parallel_ratio)
        print("sync region ratio   :", s_ratio)
        print("thread ratio        :", thread_ratio(result))
        print("sync / parallel:", s_ratio/parallel_ratio)
        ratios.append(s_ratio/parallel_ratio)
    print(ratios)
    avg_ratio = np.array(ratios).mean()
    return avg_ratio

In [35]:
get_ratio_sync_paral(test_case="./test_imbalance/")

omptracing_147.json
parllel region ratio: 0.44479910939190803
sync region ratio   : 0.0034541916037212407
thread ratio        : 0.45652589236029445
sync / parallel: 0.007765734082614332
omptracing_97.json
parllel region ratio: 0.5219847713109479
sync region ratio   : 0.022549639654060732
thread ratio        : 0.5340218810297274
sync / parallel: 0.043199803698157783
omptracing_151.json
parllel region ratio: 0.6310188415220102
sync region ratio   : 0.10792539708880057
thread ratio        : 0.6503121985459445
sync / parallel: 0.17103355714147261
omptracing_155.json
parllel region ratio: 0.6898995564531083
sync region ratio   : 0.0026827519210215135
thread ratio        : 0.7108636096718542
sync / parallel: 0.003888612329038158
omptracing_159.json
parllel region ratio: 0.667457360862941
sync region ratio   : 0.01640387626007101
thread ratio        : 0.6881097784553781
sync / parallel: 0.024576665450003875
[0.007765734082614332, 0.043199803698157783, 0.17103355714147261, 0.003888612329038158

0.05009287454025736

In [34]:
get_ratio_sync_paral(test_case="./test_balance/")

omptracing_125.json
parllel region ratio: 0.7889482204560708
sync region ratio   : 0.004988363057764883
thread ratio        : 0.7982103495467909
sync / parallel: 0.006322801583710067
omptracing_136.json
parllel region ratio: 0.8171424084991362
sync region ratio   : 0.00298072438989545
thread ratio        : 0.8265447992279191
sync / parallel: 0.0036477416407382566
omptracing_121.json
parllel region ratio: 0.8520059472159062
sync region ratio   : 0.024390380745948306
thread ratio        : 0.8624190668244205
sync / parallel: 0.028627007623184535
omptracing_129.json
parllel region ratio: 0.8658235831533347
sync region ratio   : 0.004193063430119286
thread ratio        : 0.8756935785523845
sync / parallel: 0.004842861192170493
omptracing_140.json
parllel region ratio: 0.8337759411953456
sync region ratio   : 0.00112430238875817
thread ratio        : 0.8426346488390005
sync / parallel: 0.001348446666794331
[0.006322801583710067, 0.0036477416407382566, 0.028627007623184535, 0.0048428611921704

0.008957771741319537

In [36]:
get_ratio_sync_paral(test_case="./test_imbalance_nowait/")

omptracing_113.json
parllel region ratio: 0.529235967166539
sync region ratio   : 0.0021362920162150724
thread ratio        : 0.5375300130258963
sync / parallel: 0.004036558640661753
omptracing_179.json
parllel region ratio: 0.6543534468322802
sync region ratio   : 0.0018239174672241574
thread ratio        : 0.6709312976076991
sync / parallel: 0.002787358232853708
omptracing_167.json
parllel region ratio: 0.6724316127669901
sync region ratio   : 0.0020123376035371576
thread ratio        : 0.6889664959159766
sync / parallel: 0.002992627897514493
omptracing_175.json
parllel region ratio: 0.6039757936847141
sync region ratio   : 0.0026091278155786213
thread ratio        : 0.6210874698971894
sync / parallel: 0.00431992116713974
omptracing_171.json
parllel region ratio: 0.7143346126007618
sync region ratio   : 0.0026551354013587434
thread ratio        : 0.7370876194824756
sync / parallel: 0.003716935109292661
[0.004036558640661753, 0.002787358232853708, 0.002992627897514493, 0.0043199211671

0.003570680209492471

In [37]:
get_ratio_sync_paral(test_case="./test_balance_dynamic/") # for showcase thread overhead bottleneck

omptracing_192.json
parllel region ratio: 0.9940166820763541
sync region ratio   : 6.355692606067316e-05
thread ratio        : 0.9944414318074257
sync / parallel: 6.393949639548515e-05
omptracing_196.json
parllel region ratio: 0.9939185582987473
sync region ratio   : 5.770377971005454e-05
thread ratio        : 0.9943724301435545
sync / parallel: 5.805684905292835e-05
omptracing_188.json
parllel region ratio: 0.9943431651539192
sync region ratio   : 4.203621605117283e-05
thread ratio        : 0.9947438019883947
sync / parallel: 4.227536078519315e-05
[6.393949639548515e-05, 5.805684905292835e-05, 4.227536078519315e-05]


5.475723541120222e-05

In [39]:
get_ratio_sync_paral(test_case="./test_synchronization/") # omp scan

omptracing_229.json
parllel region ratio: 0.782249484262897
sync region ratio   : 0.43853059808543843
thread ratio        : 0.9068349123407056
sync / parallel: 0.560601965111757
omptracing_223.json
parllel region ratio: 0.7810664012538985
sync region ratio   : 0.4337625543331712
thread ratio        : 0.8976714649333097
sync / parallel: 0.5553465795440988
omptracing_217.json
parllel region ratio: 0.7478479511981433
sync region ratio   : 0.4759061626232354
thread ratio        : 0.9278959705190212
sync / parallel: 0.636367542173213
[0.560601965111757, 0.5553465795440988, 0.636367542173213]


0.5841053622763562

In [38]:
# result=get_tracing(test_case+"omptracing_57.json")
# print(parallel_region_ratio(result))
# print(sync_ratio(result))
# print(thread_ratio(result))

In [16]:
df_counts,df_ratios=get_valgrind("./valgrind_sample.txt")